In [2]:
import cv2
import numpy as np

In [3]:
def Train(path, mode = "RGB"):
    img_test = cv2.imread(path)
    Mask_t = {}
    crop = None
    collet_area = []
    
#     ==============================================================================
    def mouseDraw(i, x, y, f, p):
        nonlocal collet_area
        
        if i == cv2.EVENT_LBUTTONDOWN:
            collet_area = [(x,y)]

        elif i == cv2.EVENT_LBUTTONUP:
            collet_area.append((x,y))
            cv2.rectangle(img_test, collet_area[0], collet_area[1], (255, 0, 0), 1)

    cv2.imshow('testing image', img_test)
    cv2.setMouseCallback('testing image', mouseDraw)
    cv2.waitKey()
    cv2.destroyAllWindows()

    if collet_area :
        min_x = min(collet_area[0][1], collet_area[1][1])
        max_x = max(collet_area[0][1], collet_area[1][1])
        min_y = min(collet_area[0][0], collet_area[1][0])
        max_y = max(collet_area[0][0], collet_area[1][0])
        crop = img_test[min_x : max_x , min_y : max_y]
        cv2.imshow("cutted image", crop)
        cv2.waitKey()
    else:
        print('Fault Mouse Draw')
    
    h, w, k = crop.shape
#     ==============================================================================
    if mode == "RGB":
        m = (0, ('R','G'))
        for i in range(h):
            for j in range(w):
                (R,G) = crop[i,j][0:2]
                Mask_t[(R,G)] = Mask_t.get((R,G),0) + 1

                if Mask_t[(R,G)] > m[0]:
                    m = (Mask_t[(R,G)], (R,G))

        for key in Mask_t:
            Mask_t[key] = Mask_t[key] / m[0]
#     ==============================================================================
    elif mode == "N-RGB":
        crop = crop / 255
        m = (0, ('R','G'))
        for i in range(h):
            for j in range(w):
                (R,G) = crop[i,j][0:2]
                Mask_t[(R,G)] = Mask_t.get((R,G),0) + 1

                if Mask_t[(R,G)] > m[0]:
                    m = (Mask_t[(R,G)], (R,G))

        for key in Mask_t:
            Mask_t[key] = Mask_t[key] / m[0]
#     ==============================================================================
    elif mode == "HSI":
        crop = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
        m = (0, ('',''))
        
        for i in range(h):
            for j in range(w):
                (R,G) = crop[i,j][0:2]
                Mask_t[(R,G)] = Mask_t.get((R,G),0) + 1

                if Mask_t[(R,G)] > m[0]:
                    m = (Mask_t[(R,G)], (R,G))

        for key in Mask_t:
            Mask_t[key] = Mask_t[key] / m[0]
#     ==============================================================================
#     print(Mask_t)
    cv2.destroyAllWindows()
    return Mask_t
            

In [4]:
def Test(path, Mask_t, threshold = 0.05, mode = 'HSI', title = ""):
    img_test = cv2.imread(path)

    h, w, k = img_test.shape
    output = np.full((h, w, 3), 255, dtype = np.uint8)

#     ==============================================================================
    if mode == 'RGB':
        updated_img = img_test.copy()
        
        for i in range(h):
            for j in range(w):
                (R,G) = updated_img[i,j][0:2]
                if (R,G) in Mask_t and Mask_t[(R,G)] > threshold:
                    output[i][j] = img_test[i][j]

        cv2.imshow("test_Rgb_img", output)
        cv2.imwrite('output/' + title + '.jpg', output)
        cv2.waitKey()
#     ==============================================================================
    elif mode == 'N-RGB':
        img_test1 = img_test.copy() 
        updated_img = img_test1 / 255

        for i in range(h):
            for j in range(w):
                (R,G) = updated_img[i,j][0:2]
                if (R,G) in Mask_t and Mask_t[(R,G)] > threshold:
                    output[i][j] = img_test[i][j]

        cv2.imshow("test_nRgb_img", output)
        cv2.imwrite('output/' + title + '.jpg', output)
        cv2.waitKey()
#     ==============================================================================
    elif mode == 'HSI':
        updated_img = img_test.copy()
        updated_img = cv2.cvtColor(updated_img, cv2.COLOR_BGR2HSV)

        for i in range(h):
            for j in range(w):
                (R,G) = updated_img[i,j][0:2]
                if (R,G) in Mask_t and Mask_t[(R,G)] > threshold:
                    output[i][j] = img_test[i][j]

        cv2.imshow("test_hsi_img", output)
        cv2.imwrite('output/' + title + '.jpg', output)
        cv2.waitKey()
#     ==============================================================================
    cv2.destroyAllWindows()


In [ ]:
Mask = Train('input/test_image.jpg', mode = "RGB")
Test('input/gn1.png', Mask,threshold = 0.005, mode = "RGB", title = "aaa")
# Test('input/joy1.bmp', Mask,threshold = 0.005,  mode="N-RGB", title="joy1_N-RGB_0.005_part")
# Test('input/pointer1.bmp', Mask,threshold = 0.005, mode="N-RGB", title="pointer1_N-RGB_0.005_part")